# first is to find significance among a subsector of days

In [1]:
# first thing that we do is to see is if there are significant
#!pip install openai
import openai
import pandas as pd

In [2]:
df_sig_news = pd.read_csv("raw_data.csv")

In [3]:
df_sig_news.head()

,Unnamed: 0,Date,Open,High,Low,Close*,Adj Close**,Volume,percentage,z-score,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,0,2008-09-17,11056.58,11057.31,10595.90,10609.66,10609.66,463200000.0,-0.040633,-3.403907,...,"b"" Thaksin's brother-in-law is elected as Thai...",b'1st anniversary of the beginning of the Saff...,b'Russian stock market falls 17%',b'Car bombs at U.S. embassy in Yemen kill 16',"b""CIA: Bin Laden doesn't oversee day-to-day al...",b'WTF? Six Brazilian politicians legally chang...,b'Tony Blair to appear on The Daily Show with ...,"b'At least 100 rapes in Karachi,Pakistan a day'",b'US footage of attack that killed Pakistani s...,"b""Potential HBOS- Lloyds merger 'bad news for ..."
1,1,2008-09-29,11139.62,11139.94,10365.45,10365.45,10365.45,385940000.0,-0.069790,-3.438281,...,b'Post-Ramadan crowd greeted with bombs of the...,b'Militias Assassinate Gay Iraqi Leaders',b' Japan transport minister quits over gaffes',b'Venezuela wants to work with Russia on nucle...,b'Aspiring chef dies hours after making ultra-...,b'American pressure on the Gulf-States to save...,b'Powerful earthquake reported north of New Ze...,b'LTTE has launched a nerve-gas attack against...,b'China running out of IP addresses ',b'EU bans food imports from China'
2,2,2008-10-02,10825.54,10825.54,10439.52,10482.85,10482.85,395330000.0,-0.032150,-2.553241,...,b'US approves Indian nuclear trade deal',b'Retailers Need to Realign Wall Street Report...,"b'Even after war, many Georgians revere Stalin...",b'Putin accuses Ukraine of aiding Georgia duri...,"b'US Senate approves Indo-US nuclear deal, end...",b'Murder Capitals of the World',b' Russia to deploy new nuclear missile',b'Capturing the database state: community phot...,"b'Life in Zimbabwe... "" a teachers monthly pay...",b'A Canadian Explains Why The American Electio...
3,3,2008-10-03,10483.96,10796.26,10310.25,10325.38,10325.38,299690000.0,-0.015022,-2.750429,...,b'Bangkok Sex Tycoon Runs For Governor',"b""Signs of a pending depression? The Economist...",b'The Jewel of Medina - out on Monday',"b""Phallic charms, buffalo amulets at heart of ...",b'An explosion in the South Ossetian capital o...,"b""Kenya's wild and dangerous west""",b'The Forgotten Wars: less than 1% of people k...,b'Israeli settlers turn on their own soldiers. ',"b""Development Doesn't Require Big Government~P...",b'The Bailing Out Continues in Europe- Belgium...
4,4,2008-10-06,10322.52,10322.76,9525.32,9955.50,9955.50,391460000.0,-0.035822,-3.313370,...,b'Chinese lawyers seeking redress for the infa...,"b'AIDS, cancer scientists scoop medicine Nobel '","b""Europe's economic super power, Germany, agre...",b'France urges Israel not to attack Iran',b'Attackers in the Congo shot and hacked to de...,b'17 year old Girl has 7 Kids ',"b""Iceland's Economy on Verge of Collapse""",b'Europe joins the bailout game',b'Iraq: The Surge Is Not Working',b'Strange Creature Found On Russian Coastline ...


In [4]:
# first is to clean the headline into a proper format
#1: headline xyz
#2: headliine xyz

In [5]:
def extract_headline(row_data):
    """
    row_data=df_sig_news.iloc[row_index, :]
    """
    master_string=[str(i+1)+" : "+str(row_data[10:][i]) for i in range(0, len(row_data[10:]))]
    master_string="\n".join(master_string)
    return master_string


## lets first play with openai and get the significance

In [7]:
f = open("secret_key.env.txt", "r")
secret_key=f.read()
openai.api_key=secret_key

In [8]:
#lets first look at a toy example
completion = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt="Hi there! What are you doing")
completion

<OpenAIObject text_completion id=cmpl-94CJNrswhA6MgZsHwN77miZ9NM61h at 0x10efb37c0> JSON: {
  "id": "cmpl-94CJNrswhA6MgZsHwN77miZ9NM61h",
  "object": "text_completion",
  "created": 1710787665,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": " for the new year?\n\nI am an AI digital assistant, so I am",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 7,
    "completion_tokens": 16,
    "total_tokens": 23
  }
}

In [10]:
response=completion.choices[0].text
print(response)

 for the new year?

I am an AI digital assistant, so I am


In [34]:
#by the way that this is the model
#https://medium.com/@iryna230520/first-steps-in-langchain-the-ultimate-guide-for-beginners-part-1-2baf5a4e1b81

In [11]:
def openai_prompt(prompt):
    completion = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt=prompt)
    response=completion.choices[0].text
    return response



In [12]:
start_prompt="based on the below list of headline news, is there anything significant that can \
potentially impact the US stock market"
end_prompt="for the headlines that impact the market, make 3-5 words\
summary on the headline, seperate each factor by ':'. Reply 'NO' if there isnt any significant headline"

In [13]:
keyword_list=[]

for i in range(0, len(df_sig_news)):
    print ("i: ", i)
    mid_prompt=extract_headline(row_data=df_sig_news.iloc[i, :])
    entire_prompt=start_prompt+"\n\n"+mid_prompt+"\n\n"+end_prompt
    key_list=openai_prompt(entire_prompt)
    key_list=key_list.strip().split(":")
    key_list=[x.strip() for x in key_list if len(x)>5]
    print ("keyword for ", i, " is ",key_list )
    keyword_list=keyword_list+key_list



i:  0
keyword for  0  is  ['US embassy attacked\n\nPakistan', 'Suspected US missile strike']
i:  1


KeyboardInterrupt: 

In [14]:
keyword_list = pd.read_csv("openai_raw_keyword_extraction.csv", index_col=None)
keyword_list = keyword_list['0'].tolist()

In [15]:
print(keyword_list)
len(keyword_list)

['that can potentially impact the US Stock Market\n\nAttack on US embassy', 'Yemen', 'Bank & Insurance Bailout', '.\n\nUS strikes', 'kill 20', 'Pakistan,  Bailouts', 'UN Ranking', 'Food Hoarding', 'Currency Crisis', 'Russia-US Conflict', 'NO\nFraud Indictment', 'Security', 'that can affect the US stock market\n\nNO', 'U.K. Uses Anti-Terrorism Law to Seize Icelandic', 'Icelandic Currency', 'Value Plunge', 'NO\nIceland', 'for the stock market\n\nChina democracy', 'NO,\nCFR Member', 'that may impact the stock markets\n\nLegalize Pot', 'Taxes; Worl', 'that may impact the stock market\n\nAsteroid', 'Near-miss', 'that could impact the US stock market \n\nNJ mayors/rabbis', 'Turkish honour killing', 'NO \nDubai Police/Netanyahu', 'North Korea War', 'NO\nOh Australia', 'NO\nHaitian', 'that can affect the market\n\nSouth Africa Cartoon Court Case', 'Israel Nuclear Weapons', 'South Africa Sale', 'Australia-Israel Diplomat', 'that has potential to impact the US stock market\n\nKenyan judges uncon

120

In [16]:
cleaned_keyword_list=[x for x in keyword_list if "no" not in x.lower()]
print (cleaned_keyword_list)
len(cleaned_keyword_list)

['that can potentially impact the US Stock Market\n\nAttack on US embassy', 'Yemen', 'Bank & Insurance Bailout', '.\n\nUS strikes', 'kill 20', 'Pakistan,  Bailouts', 'UN Ranking', 'Food Hoarding', 'Currency Crisis', 'Russia-US Conflict', 'Security', 'U.K. Uses Anti-Terrorism Law to Seize Icelandic', 'Icelandic Currency', 'Value Plunge', 'for the stock market\n\nChina democracy', 'that may impact the stock markets\n\nLegalize Pot', 'Taxes; Worl', 'that may impact the stock market\n\nAsteroid', 'Near-miss', 'that could impact the US stock market \n\nNJ mayors/rabbis', 'that can affect the market\n\nSouth Africa Cartoon Court Case', 'Israel Nuclear Weapons', 'South Africa Sale', 'Australia-Israel Diplomat', 'that has potential to impact the US stock market\n\nKenyan judges unconstitutional Islamic', 'Dropped', 'Facebook fined', 'Muslim punched', 'that can potentially impact the US stock market\nWall Street Tumble', 'Debt fears', 'FTSE 100 pay', '49% rise', 'Mexico Drug War', '50,000 Dead'

74

In [17]:
# lets clean the list of keywords up by removing repetitive ones
cleaned_keyword_list=([x.split("\n")[-1] for x in cleaned_keyword_list])
print(cleaned_keyword_list)
len(cleaned_keyword_list)
pd.DataFrame(cleaned_keyword_list).to_csv("cleaned_factor_affects_stock.csv")

['Attack on US embassy', 'Yemen', 'Bank & Insurance Bailout', 'US strikes', 'kill 20', 'Pakistan,  Bailouts', 'UN Ranking', 'Food Hoarding', 'Currency Crisis', 'Russia-US Conflict', 'Security', 'U.K. Uses Anti-Terrorism Law to Seize Icelandic', 'Icelandic Currency', 'Value Plunge', 'China democracy', 'Legalize Pot', 'Taxes; Worl', 'Asteroid', 'Near-miss', 'NJ mayors/rabbis', 'South Africa Cartoon Court Case', 'Israel Nuclear Weapons', 'South Africa Sale', 'Australia-Israel Diplomat', 'Kenyan judges unconstitutional Islamic', 'Dropped', 'Facebook fined', 'Muslim punched', 'Wall Street Tumble', 'Debt fears', 'FTSE 100 pay', '49% rise', 'Mexico Drug War', '50,000 Dead', 'Missile parts Syria', 'Obama-Israel', 'Military Aid', 'Campaign + Saudi', 'Campaign Saudi Refs', 'Ebola Outbreak', 'Wind Power', 'Rising Costs', 'Decreasing', 'Bill Gates to invest $2bn', 'Lego', 'Chinese Stock Ban', 'Kim orders military', 'Greek PM Resigns', 'Twitter kills Politwoops', 'Shanghai stock market', 'opens dow

In [18]:
master_single_words=[]
for i in range(0, len(cleaned_keyword_list)):
    list_of_words=cleaned_keyword_list[i].split(" ")
    if len(list_of_words)==1:
        list_of_words=cleaned_keyword_list[i].split("-")
    list_of_words=[x.lower() for x in list_of_words if x.isalpha() and len(x)>3]
    master_single_words=master_single_words+list_of_words

master_single_words=list(set(master_single_words))
print(len(master_single_words))
master_single_words
pd.DataFrame(master_single_words).to_csv("master_letter.csv")

master_single_words

122


['dead',
 'power',
 'pilgr',
 'kenyan',
 'ranking',
 'embassy',
 'orders',
 'costs',
 'currency',
 'court',
 'wind',
 'bailout',
 'missile',
 'value',
 'iran',
 'saudi',
 'refs',
 'tumble',
 'canadian',
 'strikes',
 'assaults',
 'food',
 'weapons',
 'punched',
 'deports',
 'bank',
 'twitter',
 'minister',
 'price',
 'cartoon',
 'dropped',
 'debt',
 'falls',
 'deutsche',
 'fears',
 'worl',
 'asteroid',
 'fined',
 'muslim',
 'crude',
 'down',
 'stock',
 'bailouts',
 'rising',
 'rise',
 'uses',
 'ftse',
 'dollar',
 'ebola',
 'legalize',
 'deal',
 'greek',
 'democracy',
 'security',
 'street',
 'resigns',
 'south',
 'military',
 'politwoops',
 'near',
 'crash',
 'shanghai',
 'putin',
 'drug',
 'miss',
 'judges',
 'outbreak',
 'unconstitutional',
 'crisis',
 'iraq',
 'parts',
 'afghan',
 'nuclear',
 'conflict',
 'china',
 'test',
 'yemen',
 'wall',
 'sell',
 'western',
 'drop',
 'syria',
 'isis',
 'africa',
 'chinese',
 'opens',
 'mexico',
 'insurance',
 'sexual',
 'hoarding',
 'decreasing'

In [19]:
#for future embedding works
important_issues=['oil', 'bank','military', 'russia', 'middle-east war', 'war', 'china economy', 'market price', 'nuclear war']

# With the signal of significance, we will start looking into all significance

In [20]:
df_news=pd.read_csv("Combined_News_DJIA.csv")
df_news['Date']=pd.to_datetime(df_news['Date'])

In [21]:
df_price=pd.read_csv("djprice.csv")
df_price.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume,percentage,z-score
0,2008-07-07,11289.19,11399.11,11120.74,11231.96,11231.96,248240000.0,NaN,NaN
1,2008-07-08,11225.03,11391.13,11174.88,11384.21,11384.21,271500000.0,0.013555,NaN
2,2008-07-09,11381.93,11418.24,11141.99,11147.44,11147.44,227100000.0,-0.020798,NaN
3,2008-07-10,11148.01,11270.30,11086.95,11229.02,11229.02,248010000.0,0.007318,NaN
4,2008-07-11,11226.17,11241.07,10977.68,11100.54,11100.54,275010000.0,-0.011442,NaN


In [22]:
df_price['Date']=pd.to_datetime(df_price['Date'])
df_price

,Date,Open,High,Low,Close*,Adj Close**,Volume,percentage,z-score
0,2008-07-07,11289.19,11399.11,11120.74,11231.96,11231.96,248240000.0,NaN,NaN
1,2008-07-08,11225.03,11391.13,11174.88,11384.21,11384.21,271500000.0,0.013555,NaN
2,2008-07-09,11381.93,11418.24,11141.99,11147.44,11147.44,227100000.0,-0.020798,NaN
3,2008-07-10,11148.01,11270.30,11086.95,11229.02,11229.02,248010000.0,0.007318,NaN
4,2008-07-11,11226.17,11241.07,10977.68,11100.54,11100.54,275010000.0,-0.011442,NaN
...,...,...,...,...,...,...,...,...,...
3770,2023-06-28,33881.38,33903.76,33755.92,33852.66,33852.66,268050000.0,-0.002184,0.583472
3771,2023-06-29,33854.57,34147.68,33828.63,34122.42,34122.42,279940000.0,0.007969,1.244567
3772,2023-06-30,34269.92,34467.35,34269.92,34407.60,34407.60,351060000.0,0.008358,1.887133
3773,2023-07-03,34369.78,34465.60,34286.54,34418.47,34418.47,157590000.0,0.000316,1.840525


In [23]:
df_master=df_price.merge(df_news, how='left', on='Date')

In [24]:
df_master
df_master=df_master.dropna()
df_master
len(df_master)/252

7.773809523809524

In [21]:
# lets add any significant and then lets check the sentiment

,Date,Open,High,Low,Close*,Adj Close**,Volume,percentage,z-score,Label,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
51,2008-09-17,11056.58,11057.31,10595.90,10609.66,10609.66,463200000.0,-0.040633,-3.403907,0.0,...,"b"" Thaksin's brother-in-law is elected as Thai...",b'1st anniversary of the beginning of the Saff...,b'Russian stock market falls 17%',b'Car bombs at U.S. embassy in Yemen kill 16',"b""CIA: Bin Laden doesn't oversee day-to-day al...",b'WTF? Six Brazilian politicians legally chang...,b'Tony Blair to appear on The Daily Show with ...,"b'At least 100 rapes in Karachi,Pakistan a day'",b'US footage of attack that killed Pakistani s...,"b""Potential HBOS- Lloyds merger 'bad news for ..."
52,2008-09-18,10609.01,11076.44,10459.44,11019.69,11019.69,488060000.0,0.038647,-1.564634,1.0,...,"b'Iraqis kills 3 contractors, city Fallujah is...",b'CIA Chief Speaks of Trying to Provoke Terror...,"b'German Authorities Wiretap Skype, Raid Homes...","b'In Yemen, a Planned Massacre of Americans Is...",b'Livni sweeps Kadima race; Olmert pledges coo...,b'Rising prices tip another 75 million towards...,"b""Sri Lanka's navy says it has fought a sea ba...",b'official poverty rates of 51.8% in Gaza and ...,b'India deploys planes with Nuclear Capability...,b'Tariq Ali: Has the U.S. Invasion of Pakistan...
53,2008-09-19,11027.51,11483.05,11026.70,11388.44,11388.44,655110000.0,0.033463,0.011810,1.0,...,b'America continues to lose world influence as...,b'Rice says U.S. will resist Russian moves aga...,b'14 men rob a coca cola factory - shoot out w...,"b'South Ossetia information warfare, two sides...",b'North Korea preparing to restart nuclear rea...,b' Venezuela expels U.S. rights group for crit...,b'Chavez expells Human Rights Watch from Venez...,b'Euthanasia advocate takes her own life.',"b""The complete list of the world's billionaires""","b""Swaziland's HIV/AIDS treatment crisis goes c..."
54,2008-09-22,11394.42,11394.58,10992.20,11015.69,11015.69,213210000.0,-0.032731,-1.549244,0.0,...,"b""All We are Saying is GIve Peace a Chance - U...",b'Dig pinpoints Stonehenge origins',"b'Almost 13,000 Chinese children sickened by t...","b""Pakistan's rulers were supposed to be in Hot...","b'Turkey scared to admit Armenian genocide, sa...","b""Change in dinner plans saved Pakistan's rule...",b'Pakistani troops fire on intruding U.S. chop...,b'Heads are starting to role in China as the t...,b'Russian ambassador: Czechs selling security ...,"b""Merchant of Death appears in Thai court, no ..."
55,2008-09-23,11015.69,11143.21,10833.94,10854.17,10854.17,204480000.0,-0.014663,-2.108707,0.0,...,b'Sarkozy calls for those behind the market cr...,"b""Russia engages in 'gangland' diplomacy as it...","b""EU patent office on strike: 'the organisatio...",b'Mystery surrounds hijacked Iranian ship',b' U.S.-led wars behind Wall Street crisis: Ah...,b'Pakistan blames US raids for hotel bombing',b'Double Darwin Award? A couple having sex on ...,b'Tourists Taken Hostage in Egypt - Relocated ...,"b'Bush, Ahmadinejad To Speak At UN '",b'Workers Allegedly Beat CEO To Death In India'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2016-06-27,17355.21,17355.21,17063.08,17140.24,17140.24,138740000.0,-0.014971,-3.342046,0.0,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
2009,2016-06-28,17190.51,17409.72,17190.51,17409.72,17409.72,112190000.0,0.015722,-1.833787,1.0,...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,

In [26]:
df_master['significance']=0

In [27]:
#just to match against keywords
for i in range(0,len(df_master)):
    master_string=extract_headline(df_master.iloc[i, :])
    if any(item in master_string.lower() for item in important_issues):
        df_master.loc[i, 'significance']=sum(item in master_string.lower() for item in important_issues)

In [28]:
df_master['significance'].describe()

count    2013.000000
mean        2.595132
std         1.150035
min         0.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         6.000000
Name: significance, dtype: float64

In [29]:
# lets decide that the threshold is going to be 4
threshold=4


In [30]:
start_prompt="based on the below list of headline news, tell me if the overall sentiment of the entire list is positive or negative"
end_prompt="for positive sentiment, please reply 1. For negative sentiment, please reply -1, otherwise, just reply 0. Do not apply anything other than -1, 1, or 0"

In [31]:
df_master['sentiment']=0
for i in range(0,len(df_master)):
    if df_master.loc[i, 'significance']>threshold:
        print ("i is ", i)
        master_string=extract_headline(df_master.iloc[i, 10:35])
        mid_prompt=master_string
        prompt=start_prompt+"\n\n"+mid_prompt+"\n\n"+end_prompt
        response=openai_prompt(prompt)
        df_master.loc[i, 'sentiment']=response
        print (response)


i is  17


-1
i is  63


KeyboardInterrupt: 

In [18]:
#df_master['sentiment'].to_csv("sample.txt")

In [ ]:
#lets read the sample, and notice the issues is inconsistency

**one solution does't work, lets head to the playground https://platform.openai.com/playground to see how it should work  20mins on langhcian issues**

**all the script for openai https://docs.google.com/document/d/1DSwrFViMLUos7eSvhUKVPptndd1SShLm03WA80PYzxk/edit?usp=sharing**

**Sequential chain model: https://python.langchain.com/docs/modules/chains/foundational/sequential_chains**

In [32]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI



In [33]:
llm = ChatOpenAI(temperature=0, openai_api_key=secret_key)
#defline model and temperature

In [34]:
template1 = """Here is a list of most popular headlines on reddit:

{headline_content}

First, select headlines that can have impact on the financial market in United States, but ignore headlines news about specific companies, unless it is banking related with very big macro impact. 
The highest impact headlines are events that are highly speculative and out of market expectations. """


In [35]:
prompt_template = PromptTemplate(input_variables=["headline_content"], template=template1)


In [36]:
chain1 = LLMChain(llm=llm, prompt=prompt_template, output_key='summarized_headline') #because we have a couple chain


In [37]:
llm2 = ChatOpenAI(temperature=0,openai_api_key=secret_key)
template2 = """Rate the above headlines sentiment from -2 (most negative) to 2 (most positive).

{summarized_headline}

The final output will be a list, each value is the sentiment score per headline. For example [-1, -2, 0, 1, 2]"""


prompt_template2 = PromptTemplate(input_variables=["summarized_headline"], template=template2)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)




In [38]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

In [39]:
reddit_headlines= """
1 : Mecca under threat: Outrage at plan to destroy the birthplace of the Prophet Mohamed and replace it with a new palace and luxury malls
2 : Russia plans long-range bomber flights near U.S. shores
3 : Northern European nations to boost military cooperation against Russia
4 : Genitalia replaced with ducklings and dolphins in Turkish schoolbooks
5 : Chinese hack U.S. Weather systems, satellite network
6 : 2030 the "far distant future" says Australian Prime Minister Tony Abbott.
7 : Opium poppy cultivation in Afghanistan has reached a record high this year, a UN report has revealed, highlighting the failure of the US-led campaign to crack down on the lucrative crop.
8 : Protesters over missing students in Mexico burn state assembly: Demonstrators set fire to the local legislature building on Wednesday in the capital of the southwestern state of Guerrero in protests over the apparent massacre of 43 students by corrupt police and thugs from drug gangs.
9 : Iraqi Christian fighters retake town
10 : Israel announces it won't cooperate with UN Gaza probe
11 : Alberta fishermen land rare fossil of never-before-seen dinosaur
12 : Russian bomber patrols to reach Gulf of Mexico
13 : Shell ignored internal warnings over Nigeria oil spills, documents suggest | Documents filed at London high court suggest Shell didnt act on advice to fix aged pipeline and underplayed magnitude of two spills in Nigeria in 2008
14 : India wins U.S. support for food scheme, ends WTO blockade
15 : Russia denies NATO accusations over troops in Ukraine
"""


In [40]:
list_of_sentiment = overall_chain.run(reddit_headlines)

print (list_of_sentiment)




> Entering new SimpleSequentialChain chain...
2 : Russia plans long-range bomber flights near U.S. shores
3 : Northern European nations to boost military cooperation against Russia
5 : Chinese hack U.S. Weather systems, satellite network
7 : Opium poppy cultivation in Afghanistan has reached a record high this year
12 : Russian bomber patrols to reach Gulf of Mexico
15 : Russia denies NATO accusations over troops in Ukraine
[2, 3, 5, 7, 12, 15]

> Finished chain.
[2, 3, 5, 7, 12, 15]


**now that we've learned about sequential chain, lets run it with all the headlines**

In [41]:
import ast

def sequential_chat_response(overall_chain, headline_content, secret_key):
    list_of_sentiment = overall_chain.run(headline_content)
    print ("list of sentiment: ", list_of_sentiment)
    #take the average of the sentiment
    try:
        list_of_sentiment=ast.literal_eval(list_of_sentiment)
        list_of_sentiment=[int(x) for x in list_of_sentiment]
        avg_sentiment=sum(list_of_sentiment)/len(list_of_sentiment)
    except:
        print ("error message not able to find")
        avg_sentiment=999
    return avg_sentiment


In [42]:
# lets try sentiment round no.2
df_master['sentiment']=0
for i in range(0,len(df_master)):
    if df_master.loc[i, 'significance']>threshold:
        print ("i is ", i)
        master_string=extract_headline(df_master.iloc[i, 10:35])
        response=sequential_chat_response(overall_chain, master_string, secret_key)
        df_master.loc[i, 'sentiment']=response


i is  17


> Entering new SimpleSequentialChain chain...
1 : b'FTSE plunges 440pts in 10 minutes as markets around the world go into freefall again'
6 : b'"Sources inside the (World ) bank confirm that servers in the institution\'s highly-restricted treasury unit were deeply penetrated with spy software last April."'
8 : b'Meanwhile, shares on the Tehran stock exchange have increased in value by 20% during the year.  (Of course, the majority of their eggs are in the oil basket....)'
12 : b'Congressional leaders Harry Reid and Nancy Pelosi urged President Bush Thursday to call an emergency meeting with the G8 to address international financial instability.'
15 : b'Pakistan in crisis on mission for US funding'
[-2, -2, 0, 1, -1]

> Finished chain.
list of sentiment:  [-2, -2, 0, 1, -1]
i is  63


> Entering new SimpleSequentialChain chain...
1 : b"we're now at 200+ banks bailed out"
3 : b'I thought GM needed money from the government?  Why are they building in China!?!'
13 : b'SEC Missin

KeyboardInterrupt: 

**this is quite slow, so lets make it async**

In [43]:
import asyncio
import time


In [44]:
async def get_sequential_chat_response(overall_chain, headline_content, secret_key):

    list_of_sentiment = await overall_chain.arun(headline_content)
    print ("list of sentiment: ", list_of_sentiment)
    #take the average of the sentiment
    try:
        list_of_sentiment=ast.literal_eval(list_of_sentiment)
        list_of_sentiment=[int(x) for x in list_of_sentiment]
        avg_sentiment=sum(list_of_sentiment)/len(list_of_sentiment)
    except:
        print ("error message not able to find")
        avg_sentiment=999
    return avg_sentiment


In [45]:
async def run_sequential_chat_response(df_master,secret_key, threshold=4):
    df_master['sentiment']=0
    tasks = []
    for i in range(0,len(df_master)):
        if df_master.loc[i, 'significance']>threshold:
            print ("i is ", i)
            master_string=extract_headline(df_master.iloc[i, 10:35]) #top 1 to top 25 columns
            task = get_sequential_chat_response(overall_chain, master_string, secret_key)
            tasks.append(task)
    
    
    results = await asyncio.gather(*tasks)
    for i, result in enumerate(results):
        df_master.loc[i, 'sentiment'] = result
    
    return df_master


df_master= await run_sequential_chat_response(df_master,secret_key)


i is  17
i is  63
i is  133
i is  165
i is  264
i is  265
i is  319
i is  324
i is  342
i is  414
i is  420
i is  422
i is  454
i is  472
i is  502
i is  548
i is  651
i is  693
i is  698
i is  723
i is  734
i is  741
i is  744
i is  797
i is  798
i is  799
i is  811
i is  819
i is  829
i is  857
i is  863
i is  883
i is  912
i is  930
i is  1089
i is  1140
i is  1144
i is  1221
i is  1222
i is  1241
i is  1246
i is  1250
i is  1276
i is  1335
i is  1336
i is  1362
i is  1363
i is  1370
i is  1383
i is  1397
i is  1418
i is  1428
i is  1441
i is  1485
i is  1490
i is  1493
i is  1496
i is  1548
i is  1572
i is  1657
i is  1669
i is  1687
i is  1690
i is  1703
i is  1718
i is  1815
i is  1862
i is  1877
i is  1888
i is  1897
i is  1928
i is  1932


> Entering new SimpleSequentialChain chain...


> Entering new SimpleSequentialChain chain...


> Entering new SimpleSequentialChain chain...


> Entering new SimpleSequentialChain chain...


> Entering new SimpleSequentialChain chain...


> 

CancelledError: 

In [46]:
# read the new df_master
df_master = pd.read_csv("df_master_full.csv")

In [71]:
#df_master.to_csv("master_sentiment.csv")

In [47]:
print(df_master['sentiment'])


0      -1.166667
1      -0.400000
2      -0.428571
3      -1.200000
4      -1.571429
          ...   
1954    0.000000
1955    0.000000
1956    0.000000
1957    0.000000
1958    0.000000
Name: sentiment, Length: 1959, dtype: float64
